In [1]:
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.7.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 194kB 5.9MB/s 
     |████████████████████████████████| 24.3MB 5.5MB/s 
     |████████████████████████████████| 11.9MB 201kB/s 
     |████████████████████████████████| 21.5MB 1.4MB/s 
     |████████████████████████████████| 235kB 11.5MB/s 
     |████████████████████████████████| 2.2MB 13.9MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-cp36-none-any.whl size=322720 sha256=18ea7f85cfe1cf5c6a046d5ad6f4214bc05dd7964fc4e356910455bad2c9e924
  Stored in directory: /root/.cache/pip/wheels/6d/47/1e/0af8ce3e21783c3e584c22502011a3367c091694eebc50a971
Successfully built torch-geometric


In [2]:
!pip install karateclub

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 1.8MB 6.0MB/s 
     |████████████████████████████████| 24.2MB 1.8MB/s 
  Created wheel for karateclub: filename=karateclub-1.0.22-cp36-none-any.whl size=93242 sha256=fb411ec9135ddfa17e5b221ba13dae7f68577158ef7f5e8c4a5c151d9984bd91
  Stored in directory: /root/.cache/pip/wheels/60/ef/0a/b9f163e1bc2fa9337ef4b183529d09c2ea78588cbc2c3753ef
Successfully built karateclub
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
!pip install test_tube

  Created wheel for test-tube: filename=test_tube-0.7.5-cp36-none-any.whl size=25357 sha256=aa7e5237ed11e3da967f83155a76996d3ee8501004f6c543323db8cbc65c2d20
  Stored in directory: /root/.cache/pip/wheels/02/52/96/c8b6f3c345cfd3284845ef50818c6996a5658006fe50e40e98
Successfully built test-tube


In [4]:
!pip install grakel

     |████████████████████████████████| 1.6MB 4.1MB/s 
     |████████████████████████████████| 163kB 26.6MB/s 


In [5]:
import pickle
import networkx as nx
import numpy as np
random_state = np.random.RandomState(0) # FIX THIS

def random_split_counts(total_number, no_of_splits):
    split_indices = np.sort(np.random.choice(total_number,no_of_splits-1, replace = False)+1)
    split_indices = np.insert(split_indices, 0, 0)
    split_indices = np.append(split_indices, total_number)
    split_counts = np.diff(split_indices)
    return split_counts

class GraphGenerator:
	def __init__(self, numClass):
		self.numClass = numClass

	def format_path(self, G, savePath, graphName, **kwargs):
		graphName = graphName.format(numNode=G.number_of_nodes(), numEdge=G.number_of_edges(), numClass=self.numClass, **kwargs)
		savePath = savePath.format(graphName=graphName, numNode=G.number_of_nodes(), numEdge=G.number_of_edges(), numClass=self.numClass, **kwargs)
		return savePath, graphName

	def save_graph(self, G: nx.Graph, savePath, graphName, **kwargs):
		savePath, graphName = self.format_path(G, savePath, graphName, **kwargs)
		G_d = nx.to_dict_of_lists(G)
		print("Saving graph to {}".format(os.path.join(savePath, graphName + ".graph")))
		pickle.dump(G_d, open(os.path.join(savePath, graphName + ".graph"), "wb"))
	
	def save_y(self, G: nx.Graph, savePath, graphName, **kwargs):
		savePath, graphName = self.format_path(G, savePath, graphName, **kwargs)
		ally = np.zeros((len(G.nodes()), self.numClass))
		for v in G.nodes():
			ally[v][G.nodes[v]['color'] - 1] = 1
		print("Saving labels to {}".format(os.path.join(savePath, graphName + ".ally")))
		pickle.dump(ally, open(os.path.join(savePath, graphName + ".ally"), "wb"))
	
	def save_nx_graph(self, G: nx.Graph, savePath, graphName, **kwargs):
		savePath, graphName = self.format_path(G, savePath, graphName, **kwargs)
		print("Pickling networkx graph to {}".format(os.path.join(savePath, graphName + ".gpickle.gz")))
		nx.write_gpickle(G, os.path.join(savePath, graphName + ".gpickle.gz"))

class MixhopGraphGenerator(GraphGenerator):
	def get_color(self, class_ratio): # Assign new node to a class
		if self.__coloriter:
			return next(self.__coloriter)
		else:
			return np.random.choice(list(range(1, len(class_ratio) + 1)), 1, False, class_ratio)[0]

	def color_weight(self, col1, col2):
		dist = abs(col1 - col2)
		dist = min(dist, len(self.classRatio) - dist)
		return self.heteroWeightsDict[dist]
	
	def get_neighbors(self, G, m, col, h):
		pr = dict()
		for v in G.nodes():
			degree_v = float(max(G.degree[v], 1)) # Degree is treated as at least 1, so that pr[:] is not all 0
			if G.nodes[v]['color'] == col:
				pr[v] = float(degree_v) * h
			else:
				pr[v] = float(degree_v) * ((1 - h) * self.color_weight(col, G.nodes[v]['color']))

		norm_pr = float(sum(pr.values()))
		if norm_pr == 0:
			return None
		else:
			for v in list(pr.keys()):
				pr[v] = float(pr[v]) / norm_pr
			neighbors = np.random.choice(list(pr.keys()), m, False, list(pr.values()))
			return neighbors

	def __init__(self, classRatio, heteroClsWeight="circularDist", **kwargs):
		super().__init__(len(classRatio))
		self.classRatio = classRatio
		self.heteroWeightsDict = dict()
		
		if heteroClsWeight == "circularDist":
			for i in range(2, self.numClass + 1):
				circularDist = min(i - 1, self.numClass - (i - 1))
				self.heteroWeightsDict[circularDist] = self.heteroWeightsDict.get(circularDist, 0) + 1

			maxDist = max(self.heteroWeightsDict.keys())
			weightSum = 0
			for dist, times in self.heteroWeightsDict.items():
				self.heteroWeightsDict[dist] = kwargs["heteroWeightsExponent"] ** (maxDist - dist)
				weightSum += self.heteroWeightsDict[dist] * times
			self.heteroWeightsDict = {dist: weight / weightSum for dist, weight in self.heteroWeightsDict.items()}
		
		elif heteroClsWeight == "uniform":
			for i in range(2, self.numClass + 1):
				circularDist = min(i - 1, self.numClass - (i - 1))
				self.heteroWeightsDict[circularDist] = 1.0 / len(range(2, self.numClass + 1))

	def generate_graph(self, n, m, m0, h):
		'''
		n: Target size for the generated network
		m: number of edges added with each new node
		m0: number of nodes to begin with
		h: homophily
		'''
		if n > 1 and np.sum(self.classRatio) == n:
			#print("Graph will be generated with size of each class exactly equal to the number specified in classRatio.")
			self.__colorlist = []
			for classID, classSize in enumerate(self.classRatio):
				self.__colorlist += [classID + 1] * int(classSize - m)
			random_state.shuffle(self.__colorlist)
			head_list = list(range(1, self.numClass + 1)) * m
			random_state.shuffle(head_list)
			self.__colorlist = head_list + self.__colorlist
			self.__coloriter = iter(self.__colorlist)
		else:
			self.__coloriter = None
		
		if m * self.numClass > m0:
			raise ValueError("Barabasi-Albert model requires m to be less or equal to m0")

		if m > n:
			raise ValueError("m > n should be satisfied")

		G = nx.Graph()

		for v in range(m0):
			next_color = self.get_color(self.classRatio)
			if v > 1:
				if h != 0 and h != 1:
					next_neighbor = v - 1
				else:
					next_n = self.get_neighbors(G, 1, next_color, h)
					if next_n is not None:
						next_neighbor = next_n[0]
					else:
						next_neighbor = None

			G.add_node(v, color=next_color)
			if v > 1 and next_neighbor is not None:
				G.add_edge(v, next_neighbor)
			
		for v in range(m0, n):
			if v % 1000 == 0:
				print("Generating graph... Now processing v = {}".format(v))
			col = self.get_color(self.classRatio)
			us = self.get_neighbors(G, m, col, h)

			G.add_node(v, color=col)
			assert us is not None
			for u in us:
				G.add_edge(v, u)

		assert len(list(nx.selfloop_edges(G))) == 0
		return G

	def generate_graph_contaminated(self, n, m, m0, h, contamination = 1.0):
		'''
		n: Target size for the generated network
		m: number of edges added with each new node
		m0: number of nodes to begin with
		h: homophily
		'''
		if n > 1 and np.sum(self.classRatio) == n:
			#print("Graph will be generated with size of each class exactly equal to the number specified in classRatio.")
			self.__colorlist = []
			for classID, classSize in enumerate(self.classRatio):
				self.__colorlist += [classID + 1] * int(classSize - m)
			random_state.shuffle(self.__colorlist)
			head_list = list(range(1, self.numClass + 1)) * m
			random_state.shuffle(head_list)
			self.__colorlist = head_list + self.__colorlist
			self.__coloriter = iter(self.__colorlist)
		else:
			self.__coloriter = None
		
		if m * self.numClass > m0:
			raise ValueError("Barabasi-Albert model requires m to be less or equal to m0")

		if m > n:
			raise ValueError("m > n should be satisfied")

		G = nx.Graph()

		for v in range(m0):
			next_color = self.get_color(self.classRatio)
			if v > 1:
				if h != 0 and h != 1:
					next_neighbor = v - 1
				else:
					next_n = self.get_neighbors(G, 1, next_color, h)
					if next_n is not None:
						next_neighbor = next_n[0]
					else:
						next_neighbor = None

			G.add_node(v, color=next_color)
			if v > 1 and next_neighbor is not None:
				G.add_edge(v, next_neighbor)
			
		for v in range(m0, n):
			if v % 1000 == 0:
				print("Generating graph... Now processing v = {}".format(v))
			col = self.get_color(self.classRatio)

			r = np.random.uniform()
			if r < contamination/2:
				changed_h = h + 0.4
			elif r < contamination:
				changed_h = h - 0.4
			else:
				changed_h = h
			us = self.get_neighbors(G, m, col, changed_h)

			G.add_node(v, color=col)
			assert us is not None
			for u in us:
				G.add_edge(v, u)

		assert len(list(nx.selfloop_edges(G))) == 0
		return G

	def __call__(self, n, m, m0, h):
		return self.generate_graph(n, m, m0, h)

	def save_graph(self, G:nx.Graph, savePath="{graphName}", graphName="{method}-n{numNode}-h{h}-c{numClass}", **kwargs):
		super().save_graph(G, savePath, graphName, method="mixhop", **kwargs)

	def save_y(self, G:nx.Graph, savePath="{graphName}", graphName="{method}-n{numNode}-h{h}-c{numClass}", **kwargs):
		super().save_y(G, savePath, graphName, method="mixhop", **kwargs)

In [6]:
#utils.py

import numpy as np
import torch

from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx

#from mixhop_generator import MixhopGraphGenerator, random_split_counts


class SimpleGraphDataset(InMemoryDataset):
    def __init__(self, name, data_list):
        self.name = name
        self.data, self.slices = self.collate(data_list)

        self.__indices__ = None
        self.transform = None

    @property
    def num_node_labels(self):
        if self.data.x is None:
            return 0
        for i in range(self.data.x.size(1)):
            x = self.data.x[:, i:]
            if ((x == 0) | (x == 1)).all() and (x.sum(dim=1) == 1).all():
                return self.data.x.size(1) - i
        return 0

    @property
    def num_node_attributes(self):
        if self.data.x is None:
            return 0
        return self.data.x.size(1) - self.num_node_labels

    @property
    def num_edge_labels(self):
        if self.data.edge_attr is None:
            return 0
        for i in range(self.data.edge_attr.size(1)):
            if self.data.edge_attr[:, i:].sum() == self.data.edge_attr.size(0):
                return self.data.edge_attr.size(1) - i
        return 0

    @property
    def num_edge_attributes(self):
        if self.data.edge_attr is None:
            return 0
        return self.data.edge_attr.size(1) - self.num_edge_labels

    def __repr__(self):
        return '{}({})'.format(self.name, len(self))


def load_synthetic_data(num_train=500, num_test_inlier=500, num_test_outlier=25, h_inlier=0.7, h_outlier=0.3, n_min = 50, n_max = 150, no_of_tags = 5, type1 = "mixhop", type2 = "mixhop", seed = 1213):
    np.random.seed(seed)
    print('generating data')
    g_list = []
    
    for i in range(num_train+num_test_inlier):
        
        n = np.random.randint(n_min, n_max)
        
        if type1 == "mixhop":
            tag_counts = random_split_counts(n, no_of_tags)
            g = MixhopGraphGenerator(tag_counts, heteroWeightsExponent=1.0)(n, 2, 10, h_inlier)
        elif type1 == "mixhop-contaminated":
            tag_counts = random_split_counts(n, no_of_tags)
            g = MixhopGraphGenerator(tag_counts, heteroWeightsExponent=1.0).generate_graph_contaminated(n, 2, 10, h_inlier)
        elif type1 == "mixhop-disjoint":
            tag_counts_1 = random_split_counts(n//2, no_of_tags)
            g1 = MixhopGraphGenerator(tag_counts_1, heteroWeightsExponent=1.0)(n//2, 2, 10, h_inlier+0.2)
            tag_counts_2 = random_split_counts(n//2, no_of_tags)
            g2 = MixhopGraphGenerator(tag_counts_2, heteroWeightsExponent=1.0)(n//2, 2, 10, h_inlier-0.2)
            g = nx.disjoint_union(g1,g2)
            #tags = [g.nodes[v]['color'] for v in g.nodes]
        
        g = from_networkx(g)
        g.y = torch.tensor([0])

        g_list.append(g)
    
    for i in range(num_test_outlier):
        
        n = np.random.randint(n_min, n_max)
        
        if type2 == "mixhop":
            tag_counts = random_split_counts(n, no_of_tags)
            g = MixhopGraphGenerator(tag_counts, heteroWeightsExponent=1.0)(n, 2, 10, h_outlier)
        elif type2 == "mixhop-contaminated":
            tag_counts = random_split_counts(n, no_of_tags)
            g = MixhopGraphGenerator(tag_counts, heteroWeightsExponent=1.0).generate_graph_contaminated(n, 2, 10, h_outlier)
        elif type2 == "mixhop-disjoint":
            tag_counts_1 = random_split_counts(n//2, no_of_tags)
            g1 = MixhopGraphGenerator(tag_counts_1, heteroWeightsExponent=1.0)(n//2, 2, 10, h_outlier+0.2)
            tag_counts_2 = random_split_counts(n//2, no_of_tags)
            g2 = MixhopGraphGenerator(tag_counts_2, heteroWeightsExponent=1.0)(n//2, 2, 10, h_outlier-0.2)
            g = nx.disjoint_union(g1,g2)
            #tags = [g.nodes[v]['color'] for v in g.nodes]
        
        g = from_networkx(g)
        g.y = torch.tensor([1])

        g_list.append(g)
    
    # Extracting unique tags and converting to one-hot features   
    tagset = set()
    for g in g_list:
        tagset = tagset.union(set(g.color.tolist()))
        
    tagset = list(tagset)
    tag2index = {tagset[i]:i for i in range(len(tagset))}

    for g in g_list:
        g.x = torch.zeros(len(g.color.tolist()), len(tagset))
        g.x[range(len(g.color.tolist())), [tag2index[tag] for tag in g.color.tolist()]] = 1
        del g.color

    print('Maximum node tag: %d' % len(tagset))

    return SimpleGraphDataset("MIXHOP", g_list)

def mod_CH(X, nu=0.05):
    X = np.sort(X)
    
    outlier_mean = np.mean(X[int((1-nu)*len(X)):])
    for i in range(int((1-nu)*len(X)), len(X)):
        X[i] = outlier_mean
    
    labels = np.array([0]*int((1-nu)*len(X)) + [1]*(len(X) - int((1-nu)*len(X))))
    X = X.reshape(-1, 1)
    from sklearn.metrics import calinski_harabasz_score
    score = calinski_harabasz_score(X, labels)
    return score

In [21]:
#dataloader.py

import torch, os
import numpy as np
import random
from torch_geometric.datasets import TUDataset, GNNBenchmarkDataset
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader, DenseDataLoader
from torch_geometric.utils import degree
from pathlib import Path
import math
import pickle

#from utils import load_synthetic_data

DATA_PATH = 'datasets'
if not os.path.isdir(DATA_PATH):
    os.mkdir(DATA_PATH)

class NormalizedDegree(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        deg = degree(data.edge_index[0], dtype=torch.float)
        deg = (deg - self.mean) / self.std
        data.x = deg.view(-1, 1)
        return data

class DownsamplingFilter(object):
    def __init__(self, min_nodes, max_nodes, down_class, down_rate, num_classes, reverse=True, coin=np.random.default_rng()):
        super(DownsamplingFilter, self).__init__()
        # if not reverse, downsampling mentioned class, and mentioned class as anomaly class
        # if reverse, downsampling unmentioned class, and mentioned class as normal class
        self.min_nodes = min_nodes
        self.max_nodes = max_nodes
        self.down_class = down_class
        self.down_rate = down_rate
        self.reverse = reverse
        self.coin = coin
        

    def __call__(self, data):
        # step 1: filter the graph node size
        keep = (data.num_nodes <= self.max_nodes) and (data.num_nodes >= self.min_nodes)
        # for graph classification, down_rate is 1
        # downsampling only for anomaly detection, not for classification
        if self.down_rate == 1:
            return keep
        if keep:
            # step 2: downsampling class
            mentioned_class = (data.y.item() == self.down_class)
            
            anomalous_class = not mentioned_class if self.reverse else mentioned_class
            data.y.fill_(int(anomalous_class)) # anomalous class as positive

            if anomalous_class:
                if self.coin.random() > self.down_rate:
                    keep = False
        return keep

def load_data(data_name, down_class=0, down_rate=1, dense=False, classify=False, ignore_edge_weight=True, one_class_train=False, seed=1213):
    np.random.seed(seed)
    newcoin = np.random.default_rng(seed)
    torch.manual_seed(seed)
    
    if os.path.exists(data_name + "_" + str(seed) + ".pkl"):
        with open(data_name + "_" + str(seed) + ".pkl", 'rb') as f:
            dataset_raw = pickle.load(f)
    elif data_name == 'mixhop':
        NUM = 500
        dataset_raw = load_synthetic_data(num_train=NUM, num_test_inlier=NUM, num_test_outlier=int(NUM*down_rate), seed=seed)
        with open(data_name + "_" + str(seed) + ".pkl", 'wb') as f:
            pickle.dump(dataset_raw, f)
    elif data_name == 'mixhop_hard':
        NUM = 500
        dataset_raw = load_synthetic_data(num_train=NUM, num_test_inlier=NUM, num_test_outlier=int(NUM*down_rate), seed=seed, type1="mixhop-contaminated", type2="mixhop", h_inlier=0.5, h_outlier=0.5)
        with open(data_name + "_" + str(seed) + ".pkl", 'wb') as f:
            pickle.dump(dataset_raw, f)
    elif data_name == 'mixhop_disjoint':
        NUM = 500
        dataset_raw = load_synthetic_data(num_train=NUM, num_test_inlier=NUM, num_test_outlier=int(NUM*down_rate), seed=seed, type1="mixhop-disjoint", type2="mixhop", h_inlier=0.5, h_outlier=0.5)
        with open(data_name + "_" + str(seed) + ".pkl", 'wb') as f:
            pickle.dump(dataset_raw, f)
    elif data_name in ['MNIST', 'CIFAR10']:
        dataset_raw = GNNBenchmarkDataset(root=DATA_PATH, name=data_name)
    elif 'ogbg' in data_name:
        # dataset_raw = PygGraphPropPredDataset(root=DATA_PATH, name=data_name)
        pass
        # problem: OGB needs an encoder for transform the discrete feature
        # if all embeddding are learned to be the same, then it's highly possible
        # that one-class classification will not be successful
    else:
        use_node_attr = True if data_name == 'FRANKENSTEIN' else False
        dataset_raw = TUDataset(root=DATA_PATH, name=data_name, use_node_attr=use_node_attr)
    
    
    
    # downsampling 
    # 1. Get min and max node and filter them
    num_nodes_graphs = [data.num_nodes for data in dataset_raw]
    min_nodes, max_nodes = min(num_nodes_graphs), max(num_nodes_graphs)
    if max_nodes >= 10000:
        max_nodes = 10000
    #print("min nodes, max nodes:", min_nodes, max_nodes)
    #print("#labels: ", dataset_raw.num_classes)
    
    
    # 2. Calculate down_rate for multi_class dataset: h
    from copy import deepcopy
    orig_dataset = deepcopy(dataset_raw)

    inliers = [data for data in dataset_raw if data.y == down_class]
    Ni = len(inliers)
    No = len(dataset_raw) - Ni

    down_rate = 0.5*down_rate*Ni/No
    #print("down rate:", down_rate)

    filter = DownsamplingFilter(min_nodes, max_nodes, down_class, down_rate, dataset_raw.num_classes, reverse=True, coin=newcoin)
    indices = [i for i, data in enumerate(dataset_raw) if filter(data)]
    #dataset = dataset_raw[torch.tensor(indices)].shuffle()
    dataset = dataset_raw[torch.tensor(indices)]
    
    # report the proportion info of the dataset
    n = (len(dataset) + 9) // 10
    #print("Downsampled distribution of classes in dataset:")
    #labels = np.array([data.y.item() for data in dataset])
    #label_dist = ['%4d'% (labels==c).sum() for c in range(dataset.num_classes)]
    #print("Dataset: %s, Number of graphs: %d, Class distribution %s, Num of Features %d"%(
    #        data_name, len(dataset), label_dist, dataset.num_features))

    # preprocessing: do not use original edge features or weights
    if ignore_edge_weight:
        dataset.data.edge_attr = None

    # add transforms which will be conducted when draw each elements from the dataset
    if dataset.data.x is None:
        print('using degrees as labels')
        max_degree = 0
        degs = []
        for data in dataset_raw: # ATTENTION: use dataset_raw instead of downsampled version!
            degs += [degree(data.edge_index[0], dtype=torch.long)]
            max_degree = max(max_degree, degs[-1].max().item())

        if max_degree < 1000:
            dataset.transform = T.OneHotDegree(max_degree)
            # dataset.num_features = max_degree
        else:
            # dataset['num_features'] = 1
            deg = torch.cat(degs, dim=0).to(torch.float)
            mean, std = deg.mean().item(), deg.std().item()
            dataset.transform = NormalizedDegree(mean, std)
    if dense:
        if dataset.transform is None:
            dataset.transform = T.ToDense(max_nodes)
        else:
            dataset.transform = T.Compose([dataset.transform, T.ToDense(max_nodes)])

    # now let's transform in memory before feed into dataloader to save runtime
    # because there is no random transformations
    dataset_list = [data for data in dataset]
    

    # here is for anomaly detection
    if not classify and not one_class_train: 
        m = 9 
        # if data_name == 'ENZYMES': m =7
        train_dataset = dataset_list[:m*n] # 90% train
        val_dataset = dataset_list[m*n:]
        test_dataset = dataset_list

    elif not classify and one_class_train:
        indices = [i for i, data in enumerate(dataset_list) if data.y.item()==0 and newcoin.random()<0.5]
        #train_dataset = train_dataset[torch.tensor(indices)] # only keep normal class left
        train_dataset = [dataset_list[idx] for idx in indices] # only keep normal class left
        val_dataset = []
        test_dataset = [dataset_list[idx] for idx in range(len(dataset_list)) if idx not in indices]
    else:
        # 10% test, 10% vali, 80% train
        test_dataset = dataset_list[:n] 
        val_dataset = dataset_list[n:2 * n]
        train_dataset = dataset_list[2 * n:]

    return train_dataset, val_dataset, test_dataset, dataset, dataset_raw, orig_dataset


def create_loaders(data_name, batch_size=32, down_class=0, down_rate=1, dense=False, classify=False, one_class_train=True, data_seed=1213, landmark_seed=0):

    train_dataset, val_dataset, test_dataset, dataset, dataset_raw, orig_dataset = load_data(data_name, 
                                                        down_class=down_class, 
                                                        down_rate=down_rate, 
                                                        dense=dense, 
                                                        classify=classify, 
                                                        one_class_train=one_class_train, seed=data_seed)
    
    k = int(5*math.log2(len(train_dataset)))
    random.seed(landmark_seed)
    landmark_set = random.sample(train_dataset, k)

    #sizes = [g.x.shape[0] for g in landmark_set]
    #print(max(sizes))

    '''
    print("Original distribution of classes in dataset:")
    labels = np.array([data.y.item() for data in orig_dataset])
    label_dist = ['%d'% (labels==c).sum() for c in range(orig_dataset.num_classes)]
    print("Dataset: %s, Number of graphs: %d, Class distribution %s, Num of (one-hot encoded) features %d"%(
            data_name, len(orig_dataset), label_dist, orig_dataset.num_features))
    
    print("Initial distribution of classes in dataset:")
    labels = np.array([data.y.item() for data in dataset_raw])
    label_dist = ['%d'% (labels==c).sum() for c in range(dataset_raw.num_classes)]
    print("Dataset: %s, Number of graphs: %d, Class distribution %s, Num of (one-hot encoded) features %d"%(
            data_name, len(dataset_raw), label_dist, dataset_raw.num_features))
    
    print("Downsampled distribution of classes in dataset:")
    labels = np.array([data.y.item() for data in dataset])
    label_dist = ['%d'% (labels==c).sum() for c in [0,1]]
    print("Dataset: %s, Number of graphs: %d, Class distribution %s, Num of (one-hot encoded) features %d"%(
            data_name, len(dataset), label_dist, dataset.num_features))
    '''
    print("After downsampling and test-train splitting, distribution of classes:")
    labels = np.array([data.y.item() for data in train_dataset])
    label_dist = ['%d'% (labels==c).sum() for c in [0,1]]
    print("TRAIN: Number of graphs: %d, Class distribution %s"%(len(train_dataset), label_dist))
    
    #print("After downsampling and test-train splitting, distribution of classes in TEST dataset:")
    labels = np.array([data.y.item() for data in test_dataset])
    label_dist = ['%d'% (labels==c).sum() for c in [0,1]]
    print("TEST: Number of graphs: %d, Class distribution %s"%(len(test_dataset), label_dist))
    print("Number of node features: %d" %(train_dataset[0].num_features))
    
    
    Loader = DenseDataLoader if dense else DataLoader
    num_workers = 0
    train_loader = Loader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=num_workers)
    test_loader = Loader(test_dataset, batch_size=batch_size, shuffle=False,  pin_memory=True, num_workers=num_workers)
    landmark_loader = Loader(landmark_set, batch_size=batch_size, shuffle=False,  pin_memory=True, num_workers=num_workers)

    return train_loader, test_loader, landmark_loader, train_dataset[0].num_features

In [8]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx 

from karateclub import Graph2Vec, FGSD
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score, average_precision_score


from test_tube import HyperOptArgumentParser
import logging


class EmbeddingBasedGLAD:
    def __init__(self, embedder, detector, 
                       G2V_nhid=64, G2V_wl_iter=2, 
                       FGSD_hist_bins=200, 
                       IF_n_trees=100, IF_sample_ratio=0.5,
                       LOF_n_neighbors=20, LOF_n_leaf=30, **kwargs):
        embedders = {
            'Graph2Vec': Graph2Vec(wl_iterations=G2V_wl_iter, dimensions=G2V_nhid, 
                                    attributed=True, epochs=50),
            'FGSD': FGSD(hist_bins=FGSD_hist_bins, hist_range=20)
        }
        detectors = {
            'IF': IsolationForest(n_estimators=IF_n_trees, max_samples=IF_sample_ratio, contamination=0.1),
            'LOF': LocalOutlierFactor(n_neighbors=LOF_n_neighbors, leaf_size=LOF_n_leaf, contamination=0.1)
        }

        assert embedder in embedders.keys()
        assert detector in detectors.keys()

        self.embedder = embedders[embedder]
        self.detector = detectors[detector]
        self.embedder_name = embedder
        self.detector_name = detector

    def __call__(self, dataset):
        # for inference, output anomaly score
        dataset = to_networkx_dataset(dataset)
        self.embedder.fit(dataset)
        graph_embeddings = self.embedder.get_embedding()
        self.detector.fit(graph_embeddings)

        if self.detector_name == 'IF':
            anomaly_scores = -self.detector.decision_function(graph_embeddings)
        else:
            anomaly_scores = -self.detector.negative_outlier_factor_

        return anomaly_scores

    def fit(self, dataset):
        ys = torch.cat([data.y for data in dataset])
        anomaly_scores = self(dataset)

        roc_auc = roc_auc_score(ys, anomaly_scores)
        pr_auc = average_precision_score(ys, anomaly_scores)
        avg_score_normal = anomaly_scores[ys==0].mean()
        avg_score_abnormal = anomaly_scores[ys==1].mean()  

        metrics = {'roc_auc': roc_auc, 
                   'pr_auc': pr_auc, 
                   'average_score_normal': avg_score_normal,
                   'average_score_anomaly': avg_score_abnormal}
        return metrics

    @staticmethod
    def add_model_specific_args(parent_parser):
        # nfeat should be inferred from data later
        parser = HyperOptArgumentParser(strategy=parent_parser.strategy, parents=[parent_parser], add_help=False)
        # model 
        parser.add_argument('--embedder', type=str, default='Graph2Vec')
        parser.add_argument('--detector', type=str, default='LOF')
        # Graph2Vec params
        parser.add_argument('--G2V_nhid', type=int, default=128)
        parser.add_argument('--G2V_wl_iter', type=int, default=2)
        # FSGD params
        parser.add_argument('--FGSD_hist_bins', type=int, default=200)
        # IF params
        parser.add_argument('--IF_n_trees', type=int, default=100)
        parser.add_argument('--IF_sample_ratio', type=float, default=0.5)
        # LOF params
        parser.add_argument('--LOF_n_neighbors', type=int, default=20)
        parser.add_argument('--LOF_n_leaf', type=int, default=30)
        return parser


def to_networkx_dataset(dataset):
    return [to_networkx_featured(data) for data in dataset]

def to_networkx_featured(data):
    g = to_networkx(data, to_undirected=True, remove_self_loops=True)
    features = {i: fea.argmax()+1 for i, fea in enumerate(data.x)}
    nx.set_node_attributes(g, values=features, name='feature')
    largest_cc = max(nx.connected_components(g), key=len)
    largest_cc = g.subgraph(largest_cc)
    # change node index
    mapping = {old:new for new, old in enumerate(largest_cc.nodes)}
    largest_cc = nx.relabel_nodes(largest_cc, mapping)
    # option 1: remove the checking condition, and remove isolated nodes
    # option 2: for every graph, add a new node, with node feature as 0
    # and connect all nodes to this single virtual node
    
    return largest_cc # get the largest connected component

In [19]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx 

from karateclub import Graph2Vec, FGSD
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score, average_precision_score


from test_tube import HyperOptArgumentParser
import logging


class InductiveEmbeddingBasedGLAD:
    def __init__(self, embedder, detector, 
                       G2V_nhid=64, G2V_wl_iter=2, 
                       FGSD_hist_bins=200, 
                       IF_n_trees=100, IF_sample_ratio=0.5,
                       LOF_n_neighbors=20, LOF_n_leaf=30, **kwargs):
        embedders = {
            'Graph2Vec': Graph2Vec(wl_iterations=G2V_wl_iter, dimensions=G2V_nhid, 
                                    attributed=True, epochs=50),
            'FGSD': FGSD(hist_bins=FGSD_hist_bins, hist_range=20)
        }
        detectors = {
            'OCSVM': OneClassSVM(nu=0.001),
            'LOF': LocalOutlierFactor(n_neighbors=LOF_n_neighbors, leaf_size=LOF_n_leaf, novelty=True, contamination=0.1)
        }

        assert embedder in embedders.keys()
        assert detector in detectors.keys()

        self.embedder = embedders[embedder]
        self.detector = detectors[detector]
        self.embedder_name = embedder
        self.detector_name = detector

    def __call__(self, train_dataset, test_dataset, raw=False):
        # for inference, output anomaly score
        if not raw:
            train_dataset = to_networkx_dataset(train_dataset)
            self.embedder.fit(train_dataset)
            train_graph_embeddings = self.embedder.get_embedding()
            
            test_dataset = to_networkx_dataset(test_dataset)
            self.embedder.fit(test_dataset)
            test_graph_embeddings = self.embedder.get_embedding()
            
            self.detector.fit(train_graph_embeddings)

        if self.detector_name == 'OCSVM':
            anomaly_scores = -self.detector.decision_function(test_graph_embeddings)
        else:
            anomaly_scores = -self.detector.decision_function(test_graph_embeddings)

        return anomaly_scores

    def fit(self, train_dataset, test_dataset):
        test_ys = torch.cat([data.y for data in test_dataset])
        test_anomaly_scores = self(train_dataset, test_dataset)

        roc_auc = roc_auc_score(test_ys, test_anomaly_scores)
        pr_auc = average_precision_score(test_ys, test_anomaly_scores)
        avg_score_normal = test_anomaly_scores[test_ys==0].mean()
        avg_score_abnormal = test_anomaly_scores[test_ys==1].mean()

        metrics = {'roc_auc': roc_auc,
                'pr_auc': pr_auc,
                'average_score_normal': avg_score_normal,
                'average_score_anomaly': avg_score_abnormal}
        return metrics

    @staticmethod
    def add_model_specific_args(parent_parser):
        # nfeat should be inferred from data later
        parser = HyperOptArgumentParser(strategy=parent_parser.strategy, parents=[parent_parser], add_help=False)
        # model 
        parser.add_argument('--embedder', type=str, default='Graph2Vec')
        parser.add_argument('--detector', type=str, default='LOF')
        # Graph2Vec params
        parser.add_argument('--G2V_nhid', type=int, default=128)
        parser.add_argument('--G2V_wl_iter', type=int, default=2)
        # FSGD params
        parser.add_argument('--FGSD_hist_bins', type=int, default=200)
        # IF params
        parser.add_argument('--IF_n_trees', type=int, default=100)
        parser.add_argument('--IF_sample_ratio', type=float, default=0.5)
        # LOF params
        parser.add_argument('--LOF_n_neighbors', type=int, default=20)
        parser.add_argument('--LOF_n_leaf', type=int, default=30)
        return parser


def to_networkx_dataset(dataset):
    return [to_networkx_featured(data) for data in dataset]

def to_networkx_featured(data):
    g = to_networkx(data, to_undirected=True, remove_self_loops=True)
    features = {i: fea.argmax()+1 for i, fea in enumerate(data.x)}
    nx.set_node_attributes(g, values=features, name='feature')
    largest_cc = max(nx.connected_components(g), key=len)
    largest_cc = g.subgraph(largest_cc)
    # change node index
    mapping = {old:new for new, old in enumerate(largest_cc.nodes)}
    largest_cc = nx.relabel_nodes(largest_cc, mapping)
    # option 1: remove the checking condition, and remove isolated nodes
    # option 2: for every graph, add a new node, with node feature as 0
    # and connect all nodes to this single virtual node
    
    return largest_cc # get the largest connected component

In [10]:
import torch, numpy as np
from grakel.kernels import WeisfeilerLehman, VertexHistogram, Propagation, ShortestPath, PropagationAttr
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score, average_precision_score
from test_tube import HyperOptArgumentParser

class KernelBasedGLAD:
    def __init__(self, kernel, detector, labeled=True,
                       WL_iter=5, PK_bin_width=1,
                       LOF_n_neighbors=20, LOF_n_leaf=30, **kwargs):
        kernels = {
            'WL': WeisfeilerLehman(n_iter=WL_iter, normalize=True, base_graph_kernel=VertexHistogram),
            'PK': Propagation(t_max=WL_iter, w=PK_bin_width, normalize=True) if labeled else 
                  PropagationAttr(t_max=WL_iter, w=PK_bin_width, normalize=True),
        }
        detectors = {
            'OCSVM': OneClassSVM(kernel='precomputed', nu=0.1),
            'LOF': LocalOutlierFactor(n_neighbors=LOF_n_neighbors, leaf_size=LOF_n_leaf, 
                                      metric='precomputed', contamination=0.1)
        }

        assert kernel in kernels.keys()
        assert detector in detectors.keys()
        
        self.kernel = kernels[kernel]
        self.detector = detectors[detector]
        self.kernel_name = kernel
        self.detector_name = detector
        self.labeled = labeled

    def __call__(self, dataset):
        # for inference, output anomaly score
        dataset = to_grakel_dataset(dataset, self.labeled)
        kernel_matrix = self.kernel.fit_transform(dataset)
        if self.detector_name == 'OCSVM':
            self.detector.fit(kernel_matrix)
            anomaly_scores = -self.detector.decision_function(kernel_matrix)
        else:
            self.detector.fit(np.amax(kernel_matrix) - kernel_matrix)
            anomaly_scores = -self.detector.negative_outlier_factor_

        return anomaly_scores

    def fit(self, dataset):
        ys = torch.cat([data.y for data in dataset])
        anomaly_scores = self(dataset)

        roc_auc = roc_auc_score(ys, anomaly_scores)
        pr_auc = average_precision_score(ys, anomaly_scores)
        avg_score_normal = anomaly_scores[ys==0].mean()
        avg_score_abnormal = anomaly_scores[ys==1].mean()  

        metrics = {'roc_auc': roc_auc, 
                   'pr_auc': pr_auc, 
                   'average_score_normal': avg_score_normal,
                   'average_score_anomaly': avg_score_abnormal}
        return metrics
    
    def _get_kernel_matrices(self, normalize=True):
        return calculate_kernel_matrix(self.kernel, normalize=normalize)

    @staticmethod
    def add_model_specific_args(parent_parser):
        # nfeat should be inferred from data later
        parser = HyperOptArgumentParser(strategy=parent_parser.strategy, parents=[parent_parser], add_help=False)
        # model 
        parser.add_argument('--kernel', type=str, default='WL')
        parser.add_argument('--detector', type=str, default='OCSVM')
        parser.add_argument('--labeled', type=lambda x:x=='True', default=True)
        # WL and PK params
        parser.add_argument('--WL_iter', type=int, default=5)
        parser.add_argument('--PK_bin_width', type=int, default=1)
        # LOF params
        parser.add_argument('--LOF_n_neighbors', type=int, default=20)
        parser.add_argument('--LOF_n_leaf', type=int, default=30)
        return parser

def to_grakel_dataset(dataset, labeled=True):
    def to_grakel_graph(data, labeled=True):
        edges = {tuple(edge) for edge in data.edge_index.T.numpy()}
        if labeled:
            labels = {i: fea.argmax().item()+1 for i, fea in enumerate(data.x)}
        else:
            labels = {i: fea.numpy() for i, fea in enumerate(data.x)}
        return [edges, labels] 
    return [to_grakel_graph(data, labeled) for data in dataset]
    
def calculate_kernel_matrix(kernel, normalize=True):
    def pairwise_operation(x, y, kernel):
        return np.array([kernel.metric(x[t], y[t]) for t in range(kernel.t_max)])
    
    X = kernel.X
    kernel_matrices = np.zeros(shape=(kernel.t_max, len(X), len(X)))
    cache = list()
    for (i, x) in enumerate(X):
        kernel_matrices[:,i,i] = pairwise_operation(x, x, kernel)
        for (j, y) in enumerate(cache):
            kernel_matrices[:,j,i]= pairwise_operation(y, x, kernel)
        cache.append(x) 
    for i in range(kernel.t_max):
        kernel_matrices[i] = np.triu(kernel_matrices[i]) + np.triu(kernel_matrices[i], 1).T 
        
    accumulative_kernel_matrices = np.add.accumulate(kernel_matrices, 0)
      
    if normalize:
        for i in range(kernel.t_max):
            _X_diag = np.diagonal(kernel_matrices[i])
            kernel_matrices[i] = kernel_matrices[i] / np.sqrt(np.outer(_X_diag, _X_diag))
            
            _X_diag = np.diagonal(accumulative_kernel_matrices[i])
            accumulative_kernel_matrices[i] = accumulative_kernel_matrices[i] / np.sqrt(np.outer(_X_diag, _X_diag))
            
    return kernel_matrices, accumulative_kernel_matrices

In [11]:
import torch, numpy as np
from grakel.kernels import WeisfeilerLehman, VertexHistogram, Propagation, ShortestPath, PropagationAttr
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score, average_precision_score
from test_tube import HyperOptArgumentParser

class InductiveKernelBasedGLAD:
    def __init__(self, kernel, detector, labeled=True,
                WL_iter=5, PK_bin_width=1,
                LOF_n_neighbors=20, LOF_n_leaf=30, **kwargs):
        # not sure how to set nu and contamination, especially for ocsvm
        kernels = {
              'WL': WeisfeilerLehman(n_iter=WL_iter, normalize=True, base_graph_kernel=VertexHistogram),
              'PK': Propagation(t_max=WL_iter, w=PK_bin_width, normalize=True) if labeled else
              PropagationAttr(t_max=WL_iter, w=PK_bin_width, normalize=True),
              }
        detectors = {
              'OCSVM': OneClassSVM(kernel='precomputed', nu=0.001),
              'LOF': LocalOutlierFactor(n_neighbors=LOF_n_neighbors, leaf_size=LOF_n_leaf,
              metric='precomputed', novelty=True, contamination=0.1)
              }

        assert kernel in kernels.keys()
        assert detector in detectors.keys()

        self.kernel = kernels[kernel]
        self.detector = detectors[detector]
        self.kernel_name = kernel
        self.detector_name = detector
        self.labeled = labeled

    def __call__(self, train_dataset, test_dataset, raw=False):
        # for inference, output anomaly score
        if not raw:
            train_dataset = to_grakel_dataset(train_dataset, self.labeled)
            test_dataset = to_grakel_dataset(test_dataset, self.labeled)
            # get full dataset and kernel matrix
            n_train, n_test = len(train_dataset), len(test_dataset)
            dataset = train_dataset + test_dataset
            kernel_matrix = self.kernel.fit_transform(dataset)
            train_kernel_matrix = kernel_matrix[:n_train,:][:,:n_train]
            test_kernel_matrix = kernel_matrix[n_train:,:][:,:n_train]

        if self.detector_name == 'OCSVM':
            self.detector.fit(train_kernel_matrix)
            anomaly_scores = -self.detector.decision_function(test_kernel_matrix)
        else:
            self.detector.fit(np.amax(train_kernel_matrix) - train_kernel_matrix)
            anomaly_scores = -self.detector.decision_function(test_kernel_matrix)

        return anomaly_scores

    def fit(self, train_dataset, test_dataset):
        test_ys = torch.cat([data.y for data in test_dataset])
        test_anomaly_scores = self(train_dataset, test_dataset)

        roc_auc = roc_auc_score(test_ys, test_anomaly_scores)
        pr_auc = average_precision_score(test_ys, test_anomaly_scores)
        avg_score_normal = test_anomaly_scores[test_ys==0].mean()
        avg_score_abnormal = test_anomaly_scores[test_ys==1].mean()

        metrics = {'roc_auc': roc_auc,
                'pr_auc': pr_auc,
                'average_score_normal': avg_score_normal,
                'average_score_anomaly': avg_score_abnormal}
        return metrics

    @staticmethod
    def add_model_specific_args(parent_parser):
        # nfeat should be inferred from data later
        parser = HyperOptArgumentParser(strategy=parent_parser.strategy, parents=[parent_parser], add_help=False)
        # model
        parser.add_argument('--kernel', type=str, default='WL')
        parser.add_argument('--detector', type=str, default='LOF')
        parser.add_argument('--labeled', type=lambda x:x=='True', default=True)
        # WL and PK params
        parser.add_argument('--WL_iter', type=int, default=5)
        parser.add_argument('--PK_bin_width', type=int, default=1)
        # LOF params
        parser.add_argument('--LOF_n_neighbors', type=int, default=20)
        parser.add_argument('--LOF_n_leaf', type=int, default=30)
        return parser

In [17]:
import sys
import pickle
import matplotlib.pyplot as plt
import torch.optim as optim
import math
import pickle

from types import SimpleNamespace

def run_experiment_kernel(down_cls=0, down_rate=0.05, data_seed=1213, landmark_seed=100, data = "mixhop", kernel="WL", detector="LOF", layer=1):

    
    # load data
    train_loader, test_loader, landmark_loader, num_features = create_loaders(data_name=data, 
                            batch_size=1, 
                            down_class=down_cls, 
                            down_rate=down_rate, 
                            dense=False,
                            data_seed=data_seed,
                            landmark_seed=0)


    if detector=='OCSVM':
        model = InductiveKernelBasedGLAD(kernel=kernel, detector=detector, WL_iter=layer)
        results = model.fit(train_loader, test_loader)
    
    elif kernel=='LOF':
        model = KernelBasedGLAD(kernel=kernel, detector=detector, WL_iter=layer)
        results = model.fit(test_loader)

    return results
      
def run_experiment_embedding(down_cls=0, down_rate=0.05, data_seed=1213, landmark_seed=100, data = "mixhop", embedder="Graph2Vec", detector="LOF", layer=1):

    
    # load data
    train_loader, test_loader, landmark_loader, num_features = create_loaders(data_name=data, 
                            batch_size=1, 
                            down_class=down_cls, 
                            down_rate=down_rate, 
                            dense=False,
                            data_seed=data_seed,
                            landmark_seed=0)
    
    if detector=='OCSVM':
        model = InductiveEmbeddingBasedGLAD(embedder=embedder, detector=detector, G2V_wl_iter=layer, FGSD_hist_bins=layer)
        results = model.fit(train_loader, test_loader)
    
    elif kernel=='LOF':
        model = EmbeddingBasedGLAD(embedder=embedder, detector=detector, G2V_wl_iter=layer, FGSD_hist_bins=layer)
        results = model.fit(test_loader)


    

    return results


In [35]:
import numpy as np

kernel = "WL"
detector = "LOF"
print("Kernel=", kernel)
print("Detector=", detector)

for (data, down_cls) in [("mixhop", 0), ("PROTEINS", 0), ("Tox21_AhR_training", 1), ("AIDS", 0)]:
    print(data)
    all_APs = []
    all_ROC_AUCs = []
    for layer in [1,2,4,8,16]:
        print(".", end='')
        res = run_experiment_kernel(down_cls=down_cls, data=data, kernel=kernel, detector=detector, layer=layer)

        #print("layers=", layer)
        #print("AP=", res['test_pr_auc'])
        #print("ROC-AUC=", res['test_roc_auc'])
        all_APs.append(res['pr_auc'])
        all_ROC_AUCs.append(res['roc_auc'])
    
    print("Average AP = %.2f +- %.2f, Average ROC-AUC = %.2f +- %.2f" % (np.mean(all_APs), np.std(all_APs), np.mean(all_ROC_AUCs), np.std(all_ROC_AUCs)))

kernel = "WL"
detector = "OCSVM"
print("Kernel=", kernel)
print("Detector=", detector)

for (data, down_cls) in [("mixhop", 0), ("PROTEINS", 0), ("Tox21_AhR_training", 1), ("AIDS", 0)]:
    print(data)
    all_APs = []
    all_ROC_AUCs = []
    for layer in [1,2,4,8,16]:
        print(".", end='')
        res = run_experiment_kernel(down_cls=down_cls, data=data, kernel=kernel, detector=detector, layer=layer)

        #print("layers=", layer)
        #print("AP=", res['test_pr_auc'])
        #print("ROC-AUC=", res['test_roc_auc'])
        all_APs.append(res['pr_auc'])
        all_ROC_AUCs.append(res['roc_auc'])
    
    print("Average AP = %.2f +- %.2f, Average ROC-AUC = %.2f +- %.2f" % (np.mean(all_APs), np.std(all_APs), np.mean(all_ROC_AUCs), np.std(all_ROC_AUCs)))

Kernel= WL
Detector= LOF
mixhop
.....Average AP = 0.25 +- 0.17, Average ROC-AUC = 0.76 +- 0.08
PROTEINS
.....Average AP = 0.28 +- 0.07, Average ROC-AUC = 0.70 +- 0.03
Tox21_AhR_training
.....Average AP = 0.24 +- 0.04, Average ROC-AUC = 0.67 +- 0.05
AIDS
.....Average AP = 0.56 +- 0.05, Average ROC-AUC = 0.96 +- 0.01
Kernel= WL
Detector= OCSVM
mixhop
...

KeyboardInterrupt: ignored

In [22]:
import numpy as np
'''
embedder = "Graph2Vec"
detector = "LOF"
print("Embedder=", embedder)
print("Detector=", detector)

for (data, down_cls) in [("mixhop", 0), ("PROTEINS", 0), ("Tox21_AhR_training", 1), ("AIDS", 0)]:
    print(data)
    all_APs = []
    all_ROC_AUCs = []
    for layer in [1,2,4,8,16]:
        print(".", end='')
        res = run_experiment_embedding(down_cls=down_cls, data=data, embedder=embedder, detector=detector, layer=layer)

        #print("layers=", layer)
        #print("AP=", res['test_pr_auc'])
        #print("ROC-AUC=", res['test_roc_auc'])
        all_APs.append(res['pr_auc'])
        all_ROC_AUCs.append(res['roc_auc'])
    
    print("Average AP = %.2f +- %.2f, Average ROC-AUC = %.2f +- %.2f" % (np.mean(all_APs), np.std(all_APs), np.mean(all_ROC_AUCs), np.std(all_ROC_AUCs)))
'''
embedder = "Graph2Vec"
detector = "OCSVM"
print("Embedder=", embedder)
print("Detector=", detector)

for (data, down_cls) in [("AIDS", 0), ("PROTEINS", 0), ("Tox21_AhR_training", 1), ("AIDS", 0)]:
    print(data)
    all_APs = []
    all_ROC_AUCs = []
    for layer in [1,2,4,8,16]:
        print(".", end='')
        res = run_experiment_embedding(down_cls=down_cls, data=data, embedder=embedder, detector=detector, layer=layer)

        #print("layers=", layer)
        #print("AP=", res['test_pr_auc'])
        #print("ROC-AUC=", res['test_roc_auc'])
        all_APs.append(res['pr_auc'])
        all_ROC_AUCs.append(res['roc_auc'])
    
    print("Average AP = %.2f +- %.2f, Average ROC-AUC = %.2f +- %.2f" % (np.mean(all_APs), np.std(all_APs), np.mean(all_ROC_AUCs), np.std(all_ROC_AUCs)))

Embedder= Graph2Vec
Detector= OCSVM
AIDS
.After downsampling and test-train splitting, distribution of classes:
TRAIN: Number of graphs: 202, Class distribution ['202', '0']
TEST: Number of graphs: 206, Class distribution ['198', '8']
Number of node features: 38


/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:315: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")


KeyboardInterrupt: ignored

In [ ]:
for (data, down_cls) in [("saved", 0), ("PROTEINS", 0), ("NCI1", 1)]:
    for embedder in ['Graph2Vec', 'FGSD']:
        if embedder == 'Graph2Vec':
            layers = [1,2,4,8,16]
        else:
            layers = [25,50,100,200,400]
        for detector in ['LOF', 'IF']:
            for layer in layers:
                res = run_experiment_embedding(down_cls=down_cls, data=data, embedder=embedder, detector=detector, layer=layer)

                print(data)
                print("Embedder=", embedder)
                print("Detector=", detector)
                print("layers=", layer)
                print("AP=", res['pr_auc'])
                print("ROC-AUC=", res['roc_auc'])